## Google Colab Setup

**GPU Required:** Before running, enable GPU runtime:
1. Go to **Runtime → Change runtime type**
2. Select **T4 GPU** (or better)
3. Click **Save**

In [ ]:
# Clone RFantibody repository
import os
!git clone https://github.com/RosettaCommons/RFantibody.git
os.chdir("/content/RFantibody")

In [ ]:
# Download weights
!bash include/download_weights.sh

# Create /home/weights for hardcoded paths in scripts
!sudo mkdir -p /home/weights
!sudo cp /content/RFantibody/weights/* /home/weights/
!sudo cp /home/weights/RF2_ab.pt /home/weights/RFab_overall_best.pt

In [ ]:
# Setup Python environment
import os

# Uninstall conflicting Colab packages
!pip uninstall -y opencv-python opencv-python-headless opencv-contrib-python pytensor jax jaxlib 2>/dev/null

# Install PyTorch with CUDA 11.8
!pip install -q torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu118

# Install dependencies
!pip install -q hydra-core icecream opt-einsum e3nn pyrsistent

# Install DGL
!pip install -q dgl -f https://data.dgl.ai/wheels/torch-2.3/cu118/repo.html

# Install RFantibody
!pip install -q -e .

# Build USalign
os.chdir("/content/RFantibody/include/USalign")
!make
os.chdir("/content/RFantibody")

# Set PYTHONPATH
os.environ['PYTHONPATH'] = '/content/RFantibody/include/SE3Transformer:/content/RFantibody/src:/content/RFantibody/src/rfantibody'

print("Setup complete!")

# RFantibody: Structure-Based *De Novo* Antibody Design

| Step | Model | Purpose |
|------|-------|--------|
| 1 | RFdiffusion (Ab) | Generate antibody-target docks |
| 2 | ProteinMPNN | Assign sequences to CDR loops |
| 3 | RF2 (Ab) | Predict structure and validate |

### Pipeline Flow
```
Target + Framework → RFdiffusion → ProteinMPNN → RF2
```

---
## Input Configuration

Upload your PDB files and set parameters below.

In [ ]:
# ============================================================
# SET INPUT FILES
# Click the folder icon on the left sidebar
# Navigate to /content/inputs/
# Right-click → Upload → select your file
# ============================================================
import os

os.makedirs("/content/inputs", exist_ok=True)
os.makedirs("/content/outputs", exist_ok=True)

# >>> choose: "antibody" or "nanobody" <<<
MODE = ""

# >>> EDIT THIS LINE: Your uploaded target filename <<< #from the target file, right click then copy path. example "/content/inputs/target_MAGEB1.pdb"
TARGET_PDB = ""

# >>> EDIT THIS LINE: Your target's chain ID (check in ChimeraX) <<<
TARGET_CHAIN = ""

# Rename chain to T for RFantibody
with open(TARGET_PDB, 'r') as f:
      content = f.read()
content = content.replace(f' {TARGET_CHAIN} ', ' T ')
TARGET_PDB = "/content/inputs/target_T.pdb"
with open(TARGET_PDB, 'w') as f:
      f.write(content)

if MODE == "nanobody":
      FRAMEWORK_PDB = "/content/RFantibody/scripts/examples/example_inputs/h-NbBCII10.pdb"
else:
      # >>> EDIT THIS LINE: Your uploaded framework filename <<< #same steps of copy the target file but for antibody frame work
      FRAMEWORK_PDB = "/content/inputs/your_framework.pdb"

print(f"Mode: {MODE}")
print(f"Chain {TARGET_CHAIN} renamed to T")
print(f"Target: {TARGET_PDB}")
print(f"Framework: {FRAMEWORK_PDB}")

In [ ]:
# ============================================================
# DESIGN PARAMETERS
# ============================================================
#
# HOTSPOT_RES - Epitope residues on your target
# Format: [ChainResNum,ChainResNum,...]
# Example: "[T195,T197,T256]"
#
# DESIGN_LOOPS - CDR loops to design with length ranges
# For ANTIBODY:  "[L1:8-13,L2:7,L3:9-11,H1:7,H2:6,H3:5-13]"
# For NANOBODY:  "[H1:7,H2:6,H3:5-13]"
#
# NUM_DESIGNS - How many designs to generate
#
# ============================================================

# >>> EDIT THIS LINE: Enter your hotspot residues <<<
HOTSPOT_RES = ""  # e.g. "[T195,T197,T256]"

# >>> EDIT THIS LINE: Choose antibody or nanobody loops <<<
DESIGN_LOOPS = ""  # Antibody: "[L1:8-13,L2:7,L3:9-11,H1:7,H2:6,H3:5-13]"
                   # Nanobody: "[H1:7,H2:6,H3:5-13]"

# >>> EDIT THIS LINE: Enter number of designs <<<
NUM_DESIGNS =   # e.g. 5

---
## Section 1: RFdiffusion

In [ ]:
# RFdiffusion
import subprocess
import os

os.makedirs("/content/outputs", exist_ok=True)

env = os.environ.copy()
env['PYTHONPATH'] = '/content/RFantibody/include/SE3Transformer:/content/RFantibody/src/rfantibody/rfdiffusion:/content/RFantibody/src'

print("Running RFdiffusion...\n")
result = subprocess.run([
      'python', '/content/RFantibody/scripts/rfdiffusion_inference.py',
      '--config-path', '/content/RFantibody/src/rfantibody/rfdiffusion/config/inference',
      '--config-name', 'antibody',
      f'antibody.target_pdb={TARGET_PDB}',
      f'antibody.framework_pdb={FRAMEWORK_PDB}',
      'inference.ckpt_override_path=/content/RFantibody/weights/RFdiffusion_Ab.pt',
      f'ppi.hotspot_res={HOTSPOT_RES}',
      f'antibody.design_loops={DESIGN_LOOPS}',
      f'inference.num_designs={NUM_DESIGNS}',
      'inference.final_step=48',
      'diffuser.T=50',
      'inference.deterministic=True',
      'inference.output_prefix=/content/outputs/ab_des'
  ], env=env)

print(f"\n=== RFdiffusion Complete ===")
print(f"Generated files:")
for f in sorted(os.listdir("/content/outputs")):
      if f.endswith('.pdb'):
          print(f"  {f}")

---
## Section 2: ProteinMPNN

In [ ]:
# ProteinMPNN
import subprocess
import os

os.makedirs("/content/outputs/mpnn", exist_ok=True)

env = os.environ.copy()
env['PYTHONPATH'] = '/content/RFantibody/include/SE3Transformer:/content/RFantibody/src/rfantibody/rfdiffusion:/content/RFantibody/src'

print("Running ProteinMPNN...\n")
result = subprocess.run([
      'python', '/content/RFantibody/scripts/proteinmpnn_interface_design.py',
      '-pdbdir', '/content/outputs',
      '-outpdbdir', '/content/outputs/mpnn'
  ], env=env)

print(f"\n=== ProteinMPNN Complete ===")
print(f"Generated files:")
for f in sorted(os.listdir("/content/outputs/mpnn")):
      if f.endswith('.pdb'):
          print(f"  {f}")

# Show sequences
!pip install -q biopython
from Bio.PDB import PDBParser

parser = PDBParser(QUIET=True)
print("\n=== Designed Sequences ===\n")
for f in sorted(os.listdir("/content/outputs/mpnn")):
      if f.endswith('.pdb'):
          print(f"--- {f} ---")
          structure = parser.get_structure("ab", f"/content/outputs/mpnn/{f}")
          for chain in structure[0]:
              residues = [res.resname for res in chain]
              print(f"  Chain {chain.id}: {' '.join(residues)}")
          print()

---
## Section 3: RF2

**Filtering Criteria:** pAE < 10, RMSD < 2Å

In [ ]:
# RF2 with visible output
import subprocess
import os

os.makedirs("/content/outputs/rf2", exist_ok=True)

env = os.environ.copy()
env['PYTHONPATH'] = '/content/RFantibody/include/SE3Transformer:/content/RFantibody/src/rfantibody/rf2:/content/RFantibody/src'

print("Running RF2...\n")
result = subprocess.run([
      'python', '/content/RFantibody/scripts/rf2_predict.py',
      '--config-path', '/content/RFantibody/src/rfantibody/rf2/config',
      'input.pdb_dir=/content/outputs/mpnn',
      'output.pdb_dir=/content/outputs/rf2'
  ], env=env, capture_output=True, text=True)

print(result.stdout)
print(result.stderr)

print(f"\n=== RF2 Complete ===")
print(f"Generated files:")
files = [f for f in os.listdir("/content/outputs/rf2") if f.endswith('.pdb')]
if files:
    for f in sorted(files):
          print(f"  {f}")
else:
      print("  No files passed filtering (pAE < 10, RMSD < 2)")

---
## Section 4: Results

In [ ]:
# ============================================================
# DOWNLOAD RESULTS
# ============================================================
#
# Choose what to download:
#   1 = RF2 only (final validated antibodies)
#   2 = RF2 + ProteinMPNN
#   3 = All (RFdiffusion + ProteinMPNN + RF2)
#
# ============================================================

# >>> EDIT THIS LINE: Choose 1, 2, or 3 <<<
DOWNLOAD_OPTION = 1   #choose a number

import shutil
import os
from google.colab import files

os.makedirs("/content/download_temp", exist_ok=True)
count = 0

if DOWNLOAD_OPTION >= 1:
      for f in os.listdir("/content/outputs/rf2"):
          if f.endswith('.pdb'):
              shutil.copy(f"/content/outputs/rf2/{f}", "/content/download_temp/")
              count += 1
      print(f"Added {count} RF2 files")

if DOWNLOAD_OPTION >= 2:
      mpnn_count = 0
      for f in os.listdir("/content/outputs/mpnn"):
          if f.endswith('.pdb'):
              shutil.copy(f"/content/outputs/mpnn/{f}", "/content/download_temp/")
              mpnn_count += 1
      print(f"Added {mpnn_count} ProteinMPNN files")
      count += mpnn_count

if DOWNLOAD_OPTION >= 3:
      rfd_count = 0
      for f in os.listdir("/content/outputs"):
          if f.endswith('.pdb'):
              shutil.copy(f"/content/outputs/{f}", "/content/download_temp/")
              rfd_count += 1
      print(f"Added {rfd_count} RFdiffusion files")
      count += rfd_count

shutil.make_archive('/content/rfantibody_results', 'zip', '/content/download_temp')
files.download('/content/rfantibody_results.zip')

print(f"\nTotal: {count} files downloaded")
shutil.rmtree("/content/download_temp")
